In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

C:\Users\HP\anaconda3\envs\Reacon\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HP\anaconda3\envs\Reacon\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\HP\anaconda3\envs\Reacon\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv("IMDB Dataset.csv")

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['sentiment'] = df['sentiment'].astype('category').cat.codes

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [6]:
df.describe()

,sentiment
count,50000.000000
mean,0.500000
std,0.500005
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int8  
dtypes: int8(1), object(1)
memory usage: 439.6+ KB


In [8]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [9]:
X = df['review']
Y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.7 , random_state=0)

In [11]:
num_words = 10000
max_sequence_length = 200 

In [12]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['review'])
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [13]:
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

In [14]:
embedding_size = 32
rnn_units = 64

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_size, input_length=max_sequence_length))
model.add(SimpleRNN(units=rnn_units))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 16
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Epoch 1/10
2188/2188 [==============================] - 109s 49ms/step - loss: 0.6299 - accuracy: 0.6249 - val_loss: 0.5178 - val_accuracy: 0.7457
Epoch 2/10
2188/2188 [==============================] - 108s 49ms/step - loss: 0.4991 - accuracy: 0.7634 - val_loss: 0.4670 - val_accuracy: 0.7983
Epoch 3/10
2188/2188 [==============================] - 113s 52ms/step - loss: 0.4078 - accuracy: 0.8225 - val_loss: 0.4411 - val_accuracy: 0.8173
Epoch 4/10
2188/2188 [==============================] - 108s 49ms/step - loss: 0.3763 - accuracy: 0.8385 - val_loss: 0.4104 - val_accuracy: 0.8258
Epoch 5/10
2188/2188 [==============================] - 109s 50ms/step - loss: 0.4292 - accuracy: 0.8021 - val_loss: 0.6414 - val_accuracy: 0.6405
Epoch 6/10
2188/2188 [==============================] - 111s 51ms/step - loss: 0.5077 - accuracy: 0.7439 - val_loss: 0.5644 - val_accuracy: 0.7265
Epoch 7/10
2188/2188 [==============================] - 105s 48ms/step - loss: 0.4481 - accuracy: 0.7938 - val_loss: 0

In [15]:
pred = tokenizer.texts_to_sequences(["It was a good movie"])


In [16]:
new_review_padded = pad_sequences(pred, maxlen=max_sequence_length)
predictions = model.predict(new_review_padded)
predictions

1/1 [==============================] - 0s 462ms/step


array([[0.5967582]], dtype=float32)